In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [40]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata

In [41]:
# Initialize an empty list to store the airline names
airline_names = []

# Set custom User-Agent in the headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36"
}

# Loop through each letter of the alphabet (from A to Z)
for letter in range(ord('A'), ord('Z') + 1):
    l = chr(letter)  # Get the character corresponding to the current ASCII code (e.g., 'A' for 65, 'B' for 66, etc.)

    # Build the URL for the specific letter to fetch airline names starting with that letter
    url = f"https://www.airlinequality.com/review-pages/a-z-airline-reviews/#a2z-ldr-{l}"

    # Send an HTTP GET request to fetch the webpage's HTML content using the custom User-Agent in the headers
    html_text = requests.get(url, headers=headers).text

    # Parse the HTML content using BeautifulSoup with the "lxml" parser
    soup = BeautifulSoup(html_text, "lxml")

    # Create a unique identifier (URI) to locate the container for the current letter
    uri = f"a2z-ldr-{l}"

    # Find the container that holds the list of airlines starting with the current letter
    container = soup.find("div", {"id": uri})

    # Loop through each list item in the container and extract the airline name
    for items in container.find_all("li"):
        airline_names.append(items.text)

In [42]:
#airline_names

In [43]:
# to convert names of airlines into url
start = "https://www.airlinequality.com/airline-reviews/"
end = "?sortby=post_date%3ADesc&pagesize=100"
airline_url = []
for items in airline_names:
    #converting names to lower case
    items = items.lower()
    # Replace special characters with their ASCII equivalents
    items = unicodedata.normalize('NFKD', items).encode('ASCII', 'ignore').decode('utf-8')
    # Replacing space with hypen
    items = items.replace(" ", "-")
    #creating URL
    airline_url.append(start + items + "/"+end)

In [44]:
#airline_url

In [45]:
df_airline = pd.DataFrame({"Name":airline_names,"Links":airline_url})

#### Get for a specific airline only

In [47]:
#df_airline_filtered = df_airline[df_airline['Name'].str.contains('West Jet')]
df_airline_filtered = df_airline
df_airline_filtered

,Name,Links
0,AB Aviation,https://www.airlinequality.com/airline-reviews...
1,Adria Airways,https://www.airlinequality.com/airline-reviews...
2,Aegean Airlines,https://www.airlinequality.com/airline-reviews...
3,Aer Lingus,https://www.airlinequality.com/airline-reviews...
4,Aero VIP,https://www.airlinequality.com/airline-reviews...
...,...,...
567,Yangon Airways,https://www.airlinequality.com/airline-reviews...
568,Yemenia,https://www.airlinequality.com/airline-reviews...
569,Yeti Airlines,https://www.airlinequality.com/airline-reviews...
570,Zambia Airways,https://www.airlinequality.com/airline-reviews...


#### Add review image urls

In [49]:
columns = ["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Money","Recommended","Top Review Image Url"]

df_columns = ["Airline Name","Overall_Rating","Review_Title","Review Date","Verified","Review","Top Review Image Url","Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Money","Recommended"]

In [50]:
reviews = []

# Loop through each row in the 'df_airline' DataFrame
for index, row in df_airline_filtered.iterrows():
    # Send an HTTP GET request to fetch the HTML content of the airline's review page
    html = requests.get(row['Links'], headers=headers).text
    bs = BeautifulSoup(html, "html.parser")

    # Find the container that holds the airline's review information
    container = bs.find("article", {"class": "comp comp_reviews-airline querylist position-content"})

    # Check if the container exists (i.e., the review page is valid)
    if container:
        # Extract the airline's name from the 'Name' column of the DataFrame
        print(row['Name'])

        # Loop through each review article in the container and extract relevant details
        for items in container.find_all("article"):
            verified = False
            # Extract the overall rating from the review
            rating = items.find("div", {"class": "rating-10"})
            #if rating:
            #    rating = rating.text.strip()[:1]
            #else:
            #    rating = None
            if rating and rating.text.strip()[:2] == '10':
                rating = '10'
            else:
                rating = rating.text.strip()[:1] if rating and rating.text else None

            # Extract the review title from the review
            title = items.find("h2")
            if title:
                title = title.text
            else:
                title = None

            # Extract the review date from the review
            time = items.find("h3").find("time")
            if time:
                time = time.text
            else:
                time = None

            # Extract the review content from the review
            text = items.find("div", {"class": "text_content"}).text
            text = text.split("|")
            if len(text) == 1:
                review = text[0]
            else:
                if text[0] == '✅ Trip Verified ':
                    verified = True
                review = text[1]

            # Extract the review image url from the review
            first_div = items.find("div", {"class": "large_review_img"})
            if first_div and first_div.img:
                first_image_src = first_div.img['src']
                #print(first_image_src)
            else:
                #print("No image found.")
                first_image_src = None
                
            # Extract ratings for specific categories from the review
            table = items.find("table")
            tab = [None] * 13
            for item in table.find_all("tr"):
                i = 0
                for td in item.find_all("td"):
                    if i == 0:
                        condition = td.text
                        # Finding index of the given condition in the 'columns' list
                        ind = columns.index(condition)
                        i = 1
                    else:
                        # Checking if it's a rating or not
                        if td.find("span") is None:
                            value = td.text
                            tab[ind] = value
                        # In case it's a rating, counting stars filled (i.e., stars given)
                        else:
                            value = 0
                            for star in td.find_all("span", {"class": "star fill"}):
                                value += 1
                            tab[ind] = value

            # Store the extracted review data in a list named 'data'
            data = [row['Name'], rating, title, time, verified, review, first_image_src] + tab
            reviews.append(data)

# The 'reviews' list now contains all the extracted review data for different airlines.

AB Aviation
Adria Airways
Aegean Airlines
Aer Lingus
Aero VIP
Aerocaribbean
Aeroflot Russian Airlines
AeroItalia
Aerolineas Argentinas
Aeromar
Aeromexico
Aerosur
Africa World Airlines
Afriqiyah Airways
Aigle Azur
Air Algerie
Air Antilles
Air Arabia
Air Astana
Air Austral
Air Bagan
Air Belgium
Air Berlin
Air Botswana
Air Burkina
Air Busan
Air Cairo
Air Canada
Air Canada rouge
Air Caraibes
Air China
Air Corsica
Air Costa
Air Cote d'Ivoire
Air Djibouti
Air Dolomiti
Air Europa
Air France
Air Greenland
Air Iceland Connect
Air India
Air India Express
Air Italy
Air Juan
Air KBZ
Air Koryo
Air Labrador
Air Macau
Air Madagascar
Air Malawi
Air Malta
Air Mauritius
Air Mediterranee
Air Memphis
Air Moldova
Air Montenegro
Air Namibia
Air New Zealand
Air Niugini
Air North Yukon's Airline
Air Nostrum
Air Panama
Air Pegasus
Air Premia
Air Rarotonga
Air Senegal
Air Serbia
Air Seychelles
Air Tahiti Nui
Air Tanzania
Air Transat
Air Zimbabwe
AirAsia
AirAsia India
AirAsia Philippines
AirAsia X
AirAsia Zest
a

In [51]:
#reviews

In [52]:
#creating reviews dataframe
df = pd.DataFrame(reviews, columns=df_columns)

In [53]:
df.shape

(24354, 20)

In [54]:
df.sample(10)

,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
9687,EVA Air,9,"""Well trained and personable staff""",2nd February 2020,True,Manila to Taipei. Very fast and efficient fl...,None,Boeing 777-300ER,Family Leisure,Economy Class,Manila to Taipei,October 2019,3.0,4.0,4.0,3.0,NaN,NaN,5.0,yes
3288,AirAsia,1,"""There is no customer service""",10th September 2023,True,Air Asia what a joke. Their website doesn't ...,None,None,Solo Leisure,Economy Class,Kuala Lumpur to Siem Reap,September 2023,1.0,3.0,2.0,2.0,NaN,NaN,2.0,no
17377,Qatar Airways,9,"""Outstanding flight and crew""",30th November 2024,True,Outstanding flight and crew. The food and w...,None,A380,Solo Leisure,Business Class,Doha to Sydney,November 2024,5.0,5.0,5.0,4.0,1.0,NaN,4.0,yes
3350,AirAsia India,1,"""Very disappointed with AirAsia India""",15th May 2022,True,I had worst experience with Air Asia airline...,None,None,Solo Leisure,Economy Class,Delhi to Jaipur,May 2022,NaN,NaN,NaN,1.0,NaN,NaN,1.0,no
6104,Blue Panorama Airlines,5,"""impressed by this Italian airline""",29th June 2007,False,From Milan MXP to Luxor Cairo on June 11. Posi...,None,None,None,Economy Class,Milan to Luxor,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes
19618,South African Airways,8,"""okay with good service""",18th December 2018,False,São Paulo to Perth via Johannesburg. The fli...,None,None,Couple Leisure,Business Class,São Paulo to Perth via Johannesburg,November 2018,3.0,3.0,3.0,3.0,1.0,NaN,2.0,no
4074,Alaska Airlines,1,"""not impressed at all""",17th June 2023,True,"I am not impressed at all with the staff, con...",None,None,Family Leisure,Premium Economy,Seattle to Dallas,June 2023,2.0,1.0,1.0,1.0,1.0,NaN,1.0,no
24140,XL Airways France,1,"""will not fly with them again""",11th August 2019,False,My family and I traveled this summer from Au...,https://www.airlinequality.com/wp-content/uplo...,None,Family Leisure,Economy Class,Paris to New York,August 2019,1.0,3.0,2.0,1.0,1.0,2.0,1.0,no
2131,Air Macau,1,“provided same meals as economy class”,29th February 2024,False,"In February, I flew from Narita, Tokyo to Mac...",None,A321,Family Leisure,Premium Economy,Narita to Macau,February 2024,2.0,1.0,1.0,2.0,1.0,1.0,2.0,no
20614,TAAG Angola Airlines,2,"""a low standard airline""",6th October 2017,False,Cape Town to Porto return via Luanda. Of the...,None,Boeing 777,Family Leisure,Economy Class,Cape Town to Porto via Luanda,October 2017,4.0,2.0,2.0,1.0,1.0,1.0,2.0,no


In [55]:
df.to_csv("data/bronze_20250324_Airline_Reviews_withImageUrls.csv")

In [72]:
df[df['Overall_Rating'] == '10']

,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
15,Adria Airways,10,"""Flight was very comfortable""",4th May 2018,True,Frankfurt to Ljubljana. Flight was very comf...,None,None,Business,Economy Class,Frankfurt to Ljubljana,April 2018,5.0,5.0,5.0,5.0,NaN,NaN,5.0,yes
23,Adria Airways,10,"""nice and comfortable flights""",10th November 2016,False,Two very short flights with Adria Airways fr...,None,CRJ900,Business,Economy Class,LJU to VIE,October 2016,5.0,5.0,NaN,5.0,NaN,NaN,4.0,yes
31,Adria Airways,10,"""Clean and fresh aircraft """,29th July 2016,False,Tallinn to Stockholm with Adria Airways. Cle...,None,Canadair 700,Solo Leisure,Economy Class,TLL to ARN,July 2016,5.0,5.0,5.0,5.0,NaN,NaN,5.0,yes
33,Adria Airways,10,"""nice and professional service""",11th July 2016,False,Two short flights with Adria Airways (Ljublj...,None,CRJ900,Solo Leisure,Economy Class,LJU to ZRH,July 2016,5.0,5.0,NaN,5.0,NaN,NaN,5.0,yes
41,Adria Airways,10,"""staff friendly but unobtrusive""",2nd September 2015,False,Excellent flights both ways with Adria Airways...,None,A319,Family Leisure,Economy Class,Manchester to Ljubljana,August 2015,5.0,5.0,NaN,5.0,NaN,NaN,4.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24325,ZIPAIR,10,"""I recommend ZIPAIR""",6th March 2023,False,I recommend ZIPAIR. My friend and I booked a...,None,None,Solo Leisure,Economy Class,San Francisco to Tokyo,March 2023,5.0,5.0,4.0,5.0,NaN,5.0,5.0,yes
24327,ZIPAIR,10,"""Excellent experience with ZIPAIR""",28th February 2023,True,"Excellent experience with ZIPAIR, from quick...",None,None,Couple Leisure,Economy Class,Singapore to Tokyo Narita,February 2023,3.0,5.0,NaN,5.0,NaN,2.0,5.0,yes
24328,ZIPAIR,10,"""Excellent flight""",7th February 2023,True,Excellent flight. Almost empty flight. Had t...,https://www.airlinequality.com/wp-content/uplo...,Boeing 787,Couple Leisure,Economy Class,Los Angeles to Tokyo,January 2023,5.0,5.0,NaN,5.0,5.0,3.0,5.0,yes
24329,ZIPAIR,10,"""business class seats were incredibly reasonable""",1st February 2023,True,We chose Zipair because the business class se...,None,None,Couple Leisure,Business Class,Tokyo to Los Angeles,January 2023,5.0,4.0,4.0,5.0,3.0,5.0,5.0,yes
